In [346]:
import torch
import pandas as pd
import numpy as np
from google.colab import drive
from torch import nn
from torch.utils.data import TensorDataset , DataLoader
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.utils import shuffle
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [347]:
data = pd.read_csv("/content/drive/My Drive/dataset/CO2 Emissions_Canada/CO2 Emissions_Canada.csv")
data = shuffle(data)
for i in data:
  print(data[i].describe())

count     7385
unique      42
top       FORD
freq       628
Name: Make, dtype: object
count              7385
unique             2053
top       F-150 FFV 4X4
freq                 32
Name: Model, dtype: object
count            7385
unique             16
top       SUV - SMALL
freq             1217
Name: Vehicle Class, dtype: object
count    7385.000000
mean        3.160068
std         1.354170
min         0.900000
25%         2.000000
50%         3.000000
75%         3.700000
max         8.400000
Name: Engine Size(L), dtype: float64
count    7385.000000
mean        5.615030
std         1.828307
min         3.000000
25%         4.000000
50%         6.000000
75%         6.000000
max        16.000000
Name: Cylinders, dtype: float64
count     7385
unique      27
top        AS6
freq      1324
Name: Transmission, dtype: object
count     7385
unique       5
top          X
freq      3637
Name: Fuel Type, dtype: object
count    7385.000000
mean       12.556534
std         3.500274
min         4.2

In [348]:
needed_data = data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)',
                    'CO2 Emissions(g/km)']]
needed_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),CO2 Emissions(g/km)
6245,SUBCOMPACT,2.0,4,Z,9.3,7.3,8.4,197
396,COMPACT,1.6,4,X,8.9,6.8,8.0,184
2729,MID-SIZE,2.0,4,X,9.7,6.7,8.3,194
2332,COMPACT,4.4,8,Z,17.3,11.5,14.7,344
2083,MID-SIZE,2.5,4,X,9.0,6.5,7.9,182


In [349]:
replace_dic = dict()
counter = int(1)
for i in data['Vehicle Class'].unique():
  replace_dic[i] = counter
  counter+=1
counter = int(1)
for i in data['Fuel Type'].unique():
  replace_dic[i] = counter
  counter+=1


replace_dic

{'SUBCOMPACT': 1,
 'COMPACT': 2,
 'MID-SIZE': 3,
 'STATION WAGON - SMALL': 4,
 'SUV - STANDARD': 5,
 'PICKUP TRUCK - SMALL': 6,
 'SUV - SMALL': 7,
 'FULL-SIZE': 8,
 'TWO-SEATER': 9,
 'PICKUP TRUCK - STANDARD': 10,
 'STATION WAGON - MID-SIZE': 11,
 'MINICOMPACT': 12,
 'VAN - PASSENGER': 13,
 'SPECIAL PURPOSE VEHICLE': 14,
 'MINIVAN': 15,
 'VAN - CARGO': 16,
 'Z': 1,
 'X': 2,
 'E': 3,
 'D': 4,
 'N': 5}

In [350]:
needed_data = needed_data.replace(replace_dic)
needed_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),CO2 Emissions(g/km)
6245,1,2.0,4,1,9.3,7.3,8.4,197
396,2,1.6,4,2,8.9,6.8,8.0,184
2729,3,2.0,4,2,9.7,6.7,8.3,194
2332,2,4.4,8,1,17.3,11.5,14.7,344
2083,3,2.5,4,2,9.0,6.5,7.9,182


In [351]:
train_data, test_data = train_test_split(needed_data,test_size=0.2,random_state=0)

train_feature_data = train_data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)']]
train_labels = train_data['CO2 Emissions(g/km)']

test_feature_data = test_data[['Vehicle Class', 
                    'Engine Size(L)',
                    'Cylinders',
                    'Fuel Type',
                    'Fuel Consumption City (L/100 km)',
                    'Fuel Consumption Hwy (L/100 km)',
                    'Fuel Consumption Comb (L/100 km)']]
test_labels = test_data['CO2 Emissions(g/km)']
train_feature_data.head()

,Vehicle Class,Engine Size(L),Cylinders,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km)
6543,9,5.2,10,1,17.5,11.7,14.9
971,5,4.0,6,2,14.3,11.1,12.9
917,10,5.7,8,2,16.3,11.5,14.1
2291,1,2.0,4,1,11.3,7.1,9.4
1657,8,5.0,8,1,15.8,10.2,13.3


In [352]:
train_data_tensor = TensorDataset(torch.tensor(train_feature_data.to_numpy(), dtype=torch.float),torch.tensor(train_labels.to_numpy().reshape((-1,1)),  dtype=torch.float))
test_data_tensor = TensorDataset(torch.tensor(test_feature_data.to_numpy(), dtype=torch.float),torch.tensor(test_labels.to_numpy().reshape((-1,1)),  dtype=torch.float))

In [353]:
btch_size = 128
train_data_loader = DataLoader(train_data_tensor, batch_size=btch_size)
test_data_loader = DataLoader(test_data_tensor, batch_size=btch_size)
for x,y in train_data_loader:
  print(x.shape, y.shape)
  break;

torch.Size([128, 7]) torch.Size([128, 1])


In [354]:
class Co2Net(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_dim):
    super(Co2Net,self).__init__()
    self.first_layer = nn.Linear(input_dim, hidden_dim)
    self.first_act_func = nn.LeakyReLU(negative_slope= 0.3)
    self.out_layer = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    out = self.first_layer(x)
    out = self.first_act_func(out)
    out = self.out_layer(out)
    return out

In [355]:
def train(model, data_loader, loss_fn, optim):
  model.train()
  train_loss = 0
  

  for i, (x,y) in enumerate(data_loader):
    x, y = x.to(device) , y.to(device)

    y_hat = model(x)
    mse = loss_fn(y_hat, y)
    train_loss += mse.item()
    optim.zero_grad()
    mse.backward()
    optim.step()

  num_btch = len(data_loader) 
  train_mse = train_loss / num_btch
  print(f'train RMSE {sqrt(train_mse)}')

In [356]:
def test(model, data_loader, loss_fn, condition):
  model.eval()
  test_loss = 0
  result = dict()
  with torch.no_grad():
    for x,y in data_loader:
       x, y = x.to(device) , y.to(device)
       y_hat = model(x)
       mse = loss_fn(y_hat, y)
       test_loss += mse.item()
  num_btch = len(data_loader)
  test_mse = test_loss / num_btch
  print(f'test RMSE {sqrt(test_mse)}')
  return float(condition) >= float(sqrt(test_mse))


In [357]:
def use(model, data_loader):
  model.eval()
  result = np.empty(shape=0)
  real = np.empty(shape=0)
  diff = np.empty(shape=0)
  with torch.no_grad():
    for x,y in data_loader:
       x, y = x.to(device) , y.to(device)
       y_hat = model(x)
       real = np.concatenate((real, y.numpy()), axis=None)
       result = np.concatenate((result, y_hat.numpy()), axis=None)
  return pd.DataFrame ({'real':real,'model prediction':result})



In [358]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [359]:
input_dim = 7
output_dim = 1
hidden_dim = 15

In [360]:
model = Co2Net(input_dim, output_dim, hidden_dim).to(device)
print(model)

Co2Net(
  (first_layer): Linear(in_features=7, out_features=15, bias=True)
  (first_act_func): LeakyReLU(negative_slope=0.3)
  (out_layer): Linear(in_features=15, out_features=1, bias=True)
)


In [361]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [362]:
#epoch = 1000
flag = True
c = 1
while flag:
  #print(f'epoch {i+1}/{epoch}')
  print(f'epoch {c}')
  c+=1
  train(model,train_data_loader,loss_function, optimizer)
  flag = not test(model,test_data_loader, loss_function, 4)
  print(flag)
  

res = use(model,test_data_loader)
res_mean = res['model prediction'].mean()
real_mean = res['real'].mean()
rmse =((res['real'] - res['model prediction']) ** 2).mean() ** .5
print(f'predict: {res_mean}  real: {real_mean} rmse test: {rmse}')

#res
res.to_csv('/content/drive/My Drive/dataset/CO2 Emissions_Canada/result.csv')

epoch 1
train RMSE 92.85333182883329
test RMSE 24.42532688053588
True
epoch 2
train RMSE 23.85827913316157
test RMSE 23.302378894043887
True
epoch 3
train RMSE 23.21767071129631
test RMSE 22.847463964273924
True
epoch 4
train RMSE 22.827373782434627
test RMSE 23.115024924017618
True
epoch 5
train RMSE 22.455819021035087
test RMSE 22.744167823933573
True
epoch 6
train RMSE 21.93239518434013
test RMSE 21.441913354341413
True
epoch 7
train RMSE 21.43339614863167
test RMSE 20.738275976917457
True
epoch 8
train RMSE 21.06366484180031
test RMSE 20.046516619417726
True
epoch 9
train RMSE 20.820440878336594
test RMSE 19.627018656649565
True
epoch 10
train RMSE 20.68085746741657
test RMSE 19.498542339912174
True
epoch 11
train RMSE 20.647488989105376
test RMSE 19.353265528510292
True
epoch 12
train RMSE 20.67053311550309
test RMSE 19.293411384711117
True
epoch 13
train RMSE 20.65894128901705
test RMSE 19.30704721715053
True
epoch 14
train RMSE 20.6243923154302
test RMSE 19.372129953758634
True
